In [1]:
# 필요한 라이브러리 불러오기

from konlpy.tag import Mecab 

import pandas as pd
import numpy as np

In [92]:
# mecab 실행

mecab = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic") # dictation path 설정 / mecab-ko-dic이 mecab의 단어사전임

In [93]:
# 형태소 분석할 데이터 준비 - 정책 내용 데이터 불러오기 

content = pd.read_csv('using_data/policy_content.csv', encoding = 'euc-kr')

In [94]:
content.head()

,index,name,description,content,URL
0,0,국민취업지원제도,한국형 실업부조로 고용안전망 사각지대에 있는 취업취약계층에게 취업지원서비스 및 생활...,"1. 취업취약계층에 취업지원\n- 취업지원서비스를 종합적으로 제공하고, 저소득 구직...",https://www.youthcenter.go.kr/youngPlcyUnif/yo...
1,1,청년추가고용장려금 지원사업,"청년을 정규직으로 추가로 고용한 중소?중견기업에 인건비를 지원함으로써, 양질의 청년...",1. 청년 추가채용 1명당 연 최대 900만원을 3년간 지원2. 고용위기 지역 지정...,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
2,3,국민내일배움카드,국민 누구나 일자리에 도움이 되는 훈련을 받을 수 있는 평생능력개발 기반 마련을 위...,1. 국민 누구나 국민내일배움카드 신청 가능\n- 분리 운영되었던 실업자/재직자 내...,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
3,4,청년내일채움공제,"중소·중견기업 청년 근로자의 장기근속과 자산형성을 지원하기 위해 청년, 기업, 정부...",1. 청년 지원내용\n - 청년 본인이 2년간 300만원(매월 12만 5천원)을 적...,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
4,5,청년고용지원,청년고용정책 지원,"1. 지원내용 청년고용정책 심층 모니터링, 대학일자리센터 컨설팅 및 운영지원, 청년...",https://www.youthcenter.go.kr/youngPlcyUnif/yo...


#### nouns_list  
' description ' 에서 형태소 분석처리한 모든 단어 모음 (리스트)  
' description ' 의 모든 단어를 다 보고 싶을 때 사용
     
     
#### policy_nouns  
' 한 정책 ' 의 인덱스 + 형태소 분석처리한 단어 리스트의 모음 (리스트) (DF)  
' 한 정책 ' 에 어떤 단어가 들어있는지 볼 때 사용  
  

In [95]:
nouns_list = []
policy_index = []
policy_nouns = []

for i in range(len(content['description'])):
    nouns_list.extend(mecab.nouns(content['description'][i]))
    policy_index.append(content['index'][i])
    policy_nouns.append(mecab.nouns(content['description'][i]))

policy_nouns = pd.DataFrame({'index': policy_index,
                             'nouns': policy_nouns})

In [96]:
policy_nouns['nouns'][0].index('지원')

7

### 검색단어와 앞, 뒤 2단어를 추출하여 살펴보기

In [97]:
index_num = []
front_back_list = []

###### 단어를 입력하시오.##########
word = '지원'
###################################

for i in range(len(policy_nouns)):
    
    # 정책 리스트에 이 단어가 있으면
    if word in policy_nouns['nouns'][i]:
        
        # 검색단어가 제일 첫번째 순서면 실행
        if policy_nouns['nouns'][i].index(word) == 0:
            index_num.append(policy_nouns['index'][i])
            front_back_list.append(policy_nouns['nouns'][i][:3]) # 검색단어 + 뒷2단어
            
        # 검색단어가 두번째 순서면 실행    
        elif policy_nouns['nouns'][i].index(word) == 1:
            index_num.append(policy_nouns['index'][i])
            front_back_list.append(policy_nouns['nouns'][i][0:4])
            
        else:
            index_num.append(policy_nouns['index'][i])
            front_back_list.append(policy_nouns['nouns'][i][policy_nouns['nouns'][i].index(word)-2:policy_nouns['nouns'][i].index(word)+3])

df = pd.DataFrame()
df['index'] = index_num
df['words'] = front_back_list
df

,index,words
0,0,"[취업지원서비스, 생활안정, 지원, 정책]"
1,1,"[중견기업, 인건비, 지원, 양질, 청년]"
2,3,"[훈련비, 등, 지원]"
3,4,"[장기근속, 자산형성, 지원, 청년, 기업]"
4,5,"[청년고용정책, 지원]"
...,...,...
415,3060,"[인턴근무, 취업알선, 지원, 제도]"
416,3061,"[취업처, 알선, 지원, 제도]"
417,3062,"[청년근로자, 인건비, 지원, 교통복지비용, 주거정착금]"
418,3063,"[수요기반, 인력양성, 지원, 제도]"


### 한 글자로 이루어진 단어 추출

In [157]:
one_word = [i for i in nouns_list if len(i) == 1]

from collections import Counter

cnt_rank = pd.DataFrame()

cnt_rank['word'] = pd.Series(list(dict(Counter(one_word)).keys()))
cnt_rank['cnt'] = pd.Series(list(dict(Counter(one_word)).values()))

cnt_rank.sort_values(by='cnt', ascending = False)

,word,cnt
1,등,200
0,수,93
18,내,48
9,원,36
17,취,32
...,...,...
73,마,1
74,북,1
75,전,1
76,반,1


###  모든 단어들의 빈도수 추출

In [170]:
from collections import Counter

cnt_rank = pd.DataFrame()

cnt_rank['description_word'] = pd.Series(list(dict(Counter(nouns_list)).keys()))
cnt_rank['cnt'] = pd.Series(list(dict(Counter(nouns_list)).values()))

cnt_rank.sort_values(by='cnt', ascending = False)[450:460] # 내림차순으로 정렬

,description_word,cnt
21,국민,5
995,사회경험,5
735,동기,4
718,구인업체,4
1128,채용연계,4
727,도시,4
1135,심리상담,4
1255,제작,4
1181,개인별,4
1384,이행,4


### 단순하게 빈도수로 테스트

In [37]:
# 사용자 글 입력
text = input()

미취업자인데 빅데이터 전공으로 취업하고 싶어요


In [38]:
words = [i for i in mecab.nouns(text) if len(i) > 1]
words

['미취업자', '빅데이터', '전공', '취업']

In [39]:
score_list = []

for i in policy_nouns['nouns']:
    score = 0
    for j in words:
        if j in i:
            score += 1
    score_list.append(score)

example = pd.DataFrame()
example['score'] = score_list

example.sort_values(by='score', ascending = False)

,score
770,2
76,2
65,2
280,1
256,1
...,...
391,0
392,0
393,0
394,0


In [41]:
content.iloc[example.loc[example['score']  ==  max(example['score'])].index]

,index,name,description,content,URL
65,69,데이터 청년 캠퍼스(빅데이터 청년인재 양성 및 일자리 연계),빅데이터 기술을 선도하는 국내 최고의 대학이 기업 수요에 부응하는 실무 중심의 교육...,"전액 정부 지원 프로그램*\n*빅데이터 전문교육: 온라인교육(3주), 이론실습교육(...",https://www.youthcenter.go.kr/youngPlcyUnif/yo...
76,80,중소기업 정규직 프로젝트,청년 등 미취업자의 취업난과 어려운 지역중소기업의 구인난 완화 및 취업 애로계층에 ...,"인턴 채용 중소기업 : 인턴 1명당 고용장려금 300만원 지급(1·2월차, 2회 분...",https://www.youthcenter.go.kr/youngPlcyUnif/yo...
770,1006,[3차민간기업맞춤형]데이터 기반 마케터 교육 및 취업과정,사업목적 - 4차 산업혁명 시대에 대응하여 빅데이터를 분석하여 마케팅 인사이트를 도...,① 데이터를 분석하여 마케팅을 기획하는 직무 기술 교육 지원 ② 데이터 기반 마케터...,https://youth.seoul.go.kr/site/main/youth/jobN...
